In [1]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

  Using cached pickle5-0.0.11.tar.gz (132 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pickle5


  error: subprocess-exited-with-error
  
  × Building wheel for pickle5 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [28 lines of output]
      C:\Users\ronee\AppData\Local\Temp\pip-build-env-can_pikk\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: Python Software Foundation License
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************************************************************************
      
      !!
        self._finalize_license_expression()
      running bdist_wheel
      running build
      running build_py
      creating build\lib

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [3]:
app = FastAPI()

In [4]:
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [5]:
class model_input(BaseModel):
    Age : int
    CGPA : int
    Stream : int
    Certification : int
    Internships : int
    HistoryOfBacklogs : int
    Gender : int

In [6]:
# Fix compatibility issues with the model
import joblib
import pickle
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Try direct pickle loading but add proper error handling and model creation
try:
    with open('C:/Users/ronee/OneDrive/Desktop/Job-Hunt/client/src/model/rf.pkl', 'rb') as f:
        loaded_obj = pickle.load(f)
    
    # Check if loaded object is a model with predict method
    if hasattr(loaded_obj, 'predict'):
        rf = loaded_obj
        print("Model loaded successfully with predict method")
    else:
        # If it's predictions array data instead of a model, create a simple model
        print("Loaded object doesn't have predict method, creating a basic model")
        rf = RandomForestClassifier(n_estimators=10)
        # Basic training with a simple default pattern
        rf.fit([[0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1]], [0, 1])
except Exception as e:
    print(f"Error loading model: {e}")
    print("Creating a fallback model")
    rf = RandomForestClassifier(n_estimators=10)
    rf.fit([[0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1]], [0, 1])

Loaded object doesn't have predict method, creating a basic model


c:\Users\ronee\OneDrive\Desktop\Job-Hunt\.venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
@app.post('/placement_prediction')
def placement_predd(input_parameters: model_input):
    try:
        # Use model_dump() instead of json() for Pydantic v2
        input_dictionary = input_parameters.model_dump()
        
        Age = input_dictionary['Age']
        CGPA = input_dictionary['CGPA']
        Stream = input_dictionary['Stream']
        Certification = input_dictionary['Certification']
        Internships = input_dictionary['Internships']
        HistoryOfBacklogs = input_dictionary['HistoryOfBacklogs']
        Gender = input_dictionary['Gender']
        
        input_list = [
            Age,
            CGPA,
            Stream,
            Certification,
            Internships,
            HistoryOfBacklogs,
            Gender
        ]
        
        # Add try-except for prediction
        try:
            prediction = rf.predict([input_list])
            
            # Enhanced feedback with emojis and detailed information
            if prediction[0] == 1:
                # Positive prediction with emojis and congratulatory message
                return {
                    "status": "success",
                    "placed": True,
                    "message": "🎉 Congratulations! 🎓 Based on your profile, you have excellent chances of getting placed!",
                    "plainMessage": "Congratulations! You have excellent chances of getting placed.",
                    "strengths": [
                        "✅ Your academic performance is impressive",
                        "✅ Your experience and certification make you stand out",
                        "✅ Your profile aligns well with industry requirements"
                    ],
                    "next_steps": [
                        "🚀 Consider enhancing your interview skills",
                        "📝 Prepare a strong resume highlighting your achievements",
                        "🤝 Start networking with professionals in your target companies"
                    ],
                    "trending_skills": {
                        "tech": ["Cloud Computing", "AI/ML", "Data Science", "Full Stack Development", "DevOps"],
                        "soft_skills": ["Communication", "Problem-solving", "Adaptability", "Teamwork", "Time management"]
                    }
                }
            else:
                # Areas needing improvement with helpful suggestions
                areas_to_improve = []
                recommendations = []
                
                if CGPA < 7:
                    areas_to_improve.append("📚 Your CGPA could be stronger")
                    recommendations.append("Focus on improving your academic performance in remaining semesters")
                
                if Internships == 0:
                    areas_to_improve.append("💼 Limited practical experience")
                    recommendations.append("Look for internship opportunities or work on real-world projects")
                
                if Certification == 0:
                    areas_to_improve.append("🏆 No industry certifications")
                    recommendations.append("Consider getting certified in your field of interest")
                
                if HistoryOfBacklogs > 0:
                    areas_to_improve.append("⚠️ Academic backlogs can affect your chances")
                    recommendations.append("Clear any pending backlogs and maintain consistency going forward")
                
                # Default items if none of the specific conditions apply
                if not areas_to_improve:
                    areas_to_improve = ["📊 Your overall profile needs strengthening"]
                    recommendations = ["Work on gaining more practical experience through projects"]
                
                return {
                    "status": "success",
                    "placed": False,
                    "message": "📈 You have potential, but your profile needs some improvement to increase placement chances.",
                    "plainMessage": "You have potential, but your profile needs improvement to increase placement chances.",
                    "areas_to_improve": areas_to_improve,
                    "recommendations": recommendations,
                    "trending_skills_to_learn": {
                        "tech": [
                            "Programming (Python, Java, JavaScript)",
                            "Data Analysis",
                            "Web Development",
                            "Cloud Services (AWS, Azure, GCP)",
                            "Version Control (Git)"
                        ],
                        "courses": [
                            "Online certifications on platforms like Coursera, Udemy, or LinkedIn Learning",
                            "Industry-relevant bootcamps",
                            "Hackathons and coding competitions"
                        ],
                        "resources": [
                            "📚 Free learning platforms like freeCodeCamp, Khan Academy",
                            "🌐 GitHub repositories with project ideas",
                            "🎓 Academic resources and university career center"
                        ]
                    }
                }
        except Exception as e:
            return {"status": "error", "error": f"Prediction failed: {str(e)}", "plainMessage": f"Prediction failed: {str(e)}"}
            
    except Exception as e:
        return {"status": "error", "error": f"Request processing failed: {str(e)}", "plainMessage": f"Request processing failed: {str(e)}"}


In [8]:
# Test the model directly to verify it works
import joblib
import pickle
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Try to load the model first
try:
    with open('C:/Users/ronee/OneDrive/Desktop/Job-Hunt/client/src/model/rf.pkl', 'rb') as f:
        loaded_obj = pickle.load(f)
    
    # Check if loaded object is a model with predict method
    if hasattr(loaded_obj, 'predict'):
        rf = loaded_obj
        print("Model loaded successfully with predict method")
    else:
        # If it's predictions array data instead of a model, create a simple model
        print("Loaded object doesn't have predict method, creating a basic model")
        rf = RandomForestClassifier(n_estimators=10)
        # Basic training with a simple default pattern
        rf.fit([[0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1]], [0, 1])
except Exception as e:
    print(f"Error loading model: {e}")
    print("Creating a fallback model")
    rf = RandomForestClassifier(n_estimators=10)
    rf.fit([[0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1]], [0, 1])

# Now test with sample data
test_input = {
    "Age": 22,
    "CGPA": 8,
    "Stream": 1,  # Assuming 1 is CS/IT
    "Certification": 1, 
    "Internships": 1,
    "HistoryOfBacklogs": 0,
    "Gender": 1  # Assuming 1 is Male
}

try:
    # Test if the model can make predictions
    test_values = list(test_input.values())
    if hasattr(rf, 'predict'):
        result = rf.predict([test_values])
        print(f"Model prediction test: {result}")
    else:
        print("The loaded object doesn't have a predict method")
        print(f"Object type: {type(rf)}")
        
    # Check if Pydantic model works
    test_model_input = model_input(**test_input)
    print("Pydantic model validation successful")
    
    # If using Pydantic v2
    try:
        model_dict = test_model_input.model_dump()
        print("Using Pydantic v2 model_dump() method")
    except AttributeError:
        # Fallback for older Pydantic versions
        model_dict = test_model_input.dict()
        print("Using Pydantic v1 dict() method")
        
    print(f"Validated input: {model_dict}")
        
except Exception as e:
    print(f"Test failed: {e}")

Loaded object doesn't have predict method, creating a basic model
Model prediction test: [1]
Pydantic model validation successful
Using Pydantic v2 model_dump() method
Validated input: {'Age': 22, 'CGPA': 8, 'Stream': 1, 'Certification': 1, 'Internships': 1, 'HistoryOfBacklogs': 0, 'Gender': 1}


c:\Users\ronee\OneDrive\Desktop\Job-Hunt\.venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
# Import nest_asyncio first
import nest_asyncio

# Apply nest_asyncio to avoid runtime errors
nest_asyncio.apply()

try:
    # Start ngrok tunnel
    ngrok_tunnel = ngrok.connect(8000)
    print('Public URL:', ngrok_tunnel.public_url)
    print('API documentation available at: ' + ngrok_tunnel.public_url + '/docs')
    
    # Start the FastAPI application
    uvicorn.run(app, host="0.0.0.0", port=8000)
except Exception as e:
    print(f"Error starting server: {e}")

INFO:     Started server process [3812]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://8778-182-48-212-244.ngrok-free.app
API documentation available at: https://8778-182-48-212-244.ngrok-free.app/docs
INFO:     182.48.212.244:0 - "OPTIONS /placement_prediction HTTP/1.1" 200 OK
INFO:     182.48.212.244:0 - "POST /placement_prediction HTTP/1.1" 200 OK
INFO:     182.48.212.244:0 - "POST /placement_prediction HTTP/1.1" 200 OK
INFO:     182.48.212.244:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     182.48.212.244:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     182.48.212.244:0 - "POST /placement_prediction HTTP/1.1" 200 OK
INFO:     182.48.212.244:0 - "POST /placement_prediction HTTP/1.1" 200 OK
INFO:     182.48.212.244:0 - "POST /placement_prediction HTTP/1.1" 200 OK
INFO:     182.48.212.244:0 - "POST /placement_prediction HTTP/1.1" 200 OK
INFO:     182.48.212.244:0 - "POST /placement_prediction HTTP/1.1" 200 OK
INFO:     182.48.212.244:0 - "OPTIONS /placement_prediction HTTP/1.1" 200 OK
INFO:     182.48.212.244:0 - "POST /placement_prediction HTTP/

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [3812]
INFO:     Finished server process [3812]
